In [1]:
# FIRST CELL
# VERIFIED AS WORKING
import pandas as pd

# read raw data
path = "purchase_data.csv" 
raw_data_df = pd.read_csv(path, encoding="UTF-8")
raw_data_df

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46
778,778,Sisur91,7,Male,92,Final Critic,4.19


In [2]:
# SECOND CELL
# FIND PLAYER COUNT
# VERIFIED AS WORKING
# code sourced from https://stackoverflow.com/questions/15943769/how-do-i-get-the-row-count-of-a-pandas-dataframe

# This code does not work - why?
total_rows = raw_data_df["SN"].nunique()
f"Total Number of Players: {total_rows}"

'Total Number of Players: 576'

In [3]:
# THIRD CELL
# FIND PURCHASING ANALYSIS TOTALS
# VERIFIED AS WORKING AND FORMATTED CORRECTLY
# number of unique items
unique_items_list = raw_data_df["Item ID"].unique()
unique_items_count = len(unique_items_list)
print(f"Total Number of Unique Items: {unique_items_count}")

# Summary Table
summary_table = pd.DataFrame(
    {"Number of Unique Items": [unique_items_count],
     "Average Price": ['${0:.3}'.format(raw_data_df["Price"].mean())],
     "Number of Purchases": [raw_data_df["Price"].count()],
     "Total Revenue": [(raw_data_df["Price"].sum())]
    })
# print(summary_table)
summary_table["Total Revenue"] = summary_table["Total Revenue"].map('${:.2f}'.format)
summary_table

# DEBUG total revenue:  .map('${:.2f}'.format)

Total Number of Unique Items: 179


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


In [80]:
# FOURTH CELL
# FIND GENDER DEMOGRAPHICS
# VERIFIED AS WORKING AND FORMATTED CORRECTLY
# Do .unique before value_counts
gender_demo_df = pd.DataFrame(raw_data_df.groupby(["Gender"])["SN"].nunique())

print("Breakdown of Players by Gender:")
gender_demo_df

# CODE BREAKS HERE because of .format - Why???
#gender_demo_df["Percentage of Players"] = '{0:.3%}'.format(gender_demo_df["SN"] / total_rows)
gender_demo_df["Percentage of Players"] = (gender_demo_df["SN"] / total_rows).map('{:.2%}'.format)

# Formatting sourced from https://pandas.pydata.org/pandas-docs/version/0.18/generated/pandas.formats.style.Styler.format.html
#gender_demo_df.style.format({"Percentage of Players":'{0:.3%}'})
#gender_demo_df.style.format({'Percentage of Players':'{:.3%}'.format})
gender_demo_df = gender_demo_df.sort_values(["Percentage of Players"], ascending=False)
gender_demo_df

Breakdown of Players by Gender:


,SN,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [79]:
# FIFTH CELL
# PURCHASING ANALYSIS (GENDER)
# DEBUGGING CELL
# Reformat ALL columns with floating values as Percent with 2 decimal places
# default for float is 3 decimal places
# pd.options.display.float_format = '{:.2%}'.format
#gender_demo_df

# FIND PURCHASING ANALYSIS STATS BROKEN DOWN BY GENDER
# Establish first column (Purchase Count) in the data frame
#purchase_count_by_gender["Purchase Count"] = (raw_data_df["Gender"].value_counts())
purchase_count_by_gender = pd.DataFrame(raw_data_df["Gender"].value_counts())
#purchase_count_by_gender["Gender"] = (raw_data_df["Gender"].value_counts())
purhcase_count_by_gender = purchase_count_by_gender.rename(columns={"Gender":"Purchase Count"})
purchase_count_by_gender

# Establish 2nd column (Average Purchase Price) in the data frame
gender_purchases= (raw_data_df[["Gender", "Price"]])
avg_grouped_price_by_gender = gender_purchases.groupby('Gender').mean()
purchase_count_by_gender["Average Purchase Price"] = avg_grouped_price_by_gender
purchase_count_by_gender["Average Purchase Price"] = purchase_count_by_gender["Average Purchase Price"].map('${:.2f}'.format)
purchase_count_by_gender

# Establish 3rd column (Total Purchase Value) in the data frame
sum_grouped_price_by_gender = gender_purchases.groupby('Gender').sum()
purchase_count_by_gender["Total Purchase Value"] = sum_grouped_price_by_gender
# DEBUG - Invalid Syntax
#purchase_count_by_gender["Total Purchase Value"] = purchase_count_by_gender["Total Purchase Value"].map('${:.2f}'.format)
purchase_count_by_gender

#.map('${:.2f}'.format)
# Establish 4th column (Avg Total Purchase per Person) in the data frame
total_grouped_price_per_person = pd.DataFrame(raw_data_df.groupby(["SN", "Gender"]).sum())
avg_price_per_person_groued_by_gender = pd.DataFrame(total_grouped_price_per_person.groupby(["Gender"])["Price"].mean())

# DEBUG HERE - avg_price_per_person_grouped_by_gender not defined
purchase_count_by_gender["Avg Total Purchase per Person"] = avg_price_per_person_grouped_by_gender
# DEBUG - Invalid Syntax
#purchase_count_by_gender["Avg Total Purchase per Person"] = purchase_count_by_gender["Avg Total Purchase per Person"].map('${:.2f}'.format)
purchase_count_by_gender

NameError: name 'avg_price_per_person_grouped_by_gender' is not defined

In [78]:
# SIXTH CELL
# FIND AGE DEMOGRAPHICS
age_demo_df = raw_data_df[["SN","Age"]]
age_demo_df

#GROUPING SYNTAX:  new_df = old_df.groupby(['column to group by'])

age_demo_df = raw_data_df.filter(["SN","Age", "Price"], axis=1)
no_dupe_age_demo_df = age_demo_df.drop_duplicates()

# Establish bins for ages and names for the bins
bins = [0, 10, 15, 20, 25, 30, 35, 40, 1000]

# Create the names for the bins
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# Establish bins in dataframe under field "Total Count"
# DEBUG ATTEMPT change age_demo_df to no_dupe_age_demo_df
no_dupe_age_demo_df["Total Count"] = pd.cut(no_dupe_age_demo_df["Age"], bins, labels=group_names, include_lowest=True, right=False)

no_dupe_age_demo_df["Age Count"] = 1

# Count each bin
only_age_df = pd.DataFrame(no_dupe_age_demo_df["Total Count"].value_counts())

# Find Percentage of Players
only_age_df["Percentage of Players"] = (only_age_df["Total Count"]) / total_rows

# Reformat Percentage of Players
only_age_df["Percentage of Players"] = only_age_df["Percentage of Players"].map('{:.2%}'.format)

# Resort dataframe
only_age_df.sort_index()


#resorted_only_age_df = only_age_df.sort_index(axis=1, ascending=True)
#resorted_only_age_df

#only_age_demo_df = age_demo_df.groupby(['Age Bins'])
#only_age_demo_df
#age_demo_df.columns

#grouped_age_demo_df = age_demo_df.groupby(['SN'])
#grouped_age_demo_df


C:\Users\small\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\small\anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Total Count,Percentage of Players
<10,23,3.99%
10-14,28,4.86%
15-19,135,23.44%
20-24,365,63.37%
25-29,101,17.53%
30-34,73,12.67%
35-39,41,7.12%
40+,13,2.26%


In [77]:
# SEVENTH CELL
# FIND PURCHASING STATS BY AGE BIN

# Start with all rows of purchases with just the columns needed
age_demo_df

# Establish bins for ages and names for the bins
bins = [0, 10, 15, 20, 25, 30, 35, 40, 1000]

# Create the names for the bins
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# Establish bins in dataframe under field "Total Count"
purchase_demo_df = age_demo_df
purchase_demo_df["Purchase Counts"] = pd.cut(age_demo_df["Age"], bins, labels=group_names, include_lowest=True, right=False)
purchase_demo_df

# Count each bin
purchase_stats_df = pd.DataFrame(purchase_demo_df["Purchase Counts"].value_counts())

# Find Average Purchase Price for each bin
average_purch_price_age = purchase_demo_df

# Resort dataframe
purchase_stats_df = purchase_stats_df.sort_index()
print(purchase_stats_df)

# Create dataframe with Average Purchase Price for each Age Bin
average_purchase_df = purchase_demo_df.groupby("Purchase Counts").mean()
average_purchase_df

# Create dataframe with Total Purchase Price for each Age Bin
# DEBUG - Should this be "Purchase Counts or "Price"???
total_purchase_df = purchase_demo_df.groupby("Purchase Counts").sum()
total_purchase_df

# Create dataframe with Average Purchase Total per Person for each Age Bin
# Take purchase_demo_df and group by(SN) and sum
# Create age bins again
# Then group by (age bin) and perform mean
average_purch_total_df = purchase_demo_df.groupby("SN").sum()

# Establish bins for ages and names for the bins - for average_purch_total_df
bins = [0, 10, 15, 20, 25, 30, 35, 40, 1000]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# Establish bins in dataframe under field "Total Count"
average_purch_total_df["Age Bins"] = pd.cut(average_purch_total_df["Age"], bins, labels=group_names, include_lowest=True, right=False)
average_purch_total_df

# Then group by (age bin) and perform mean
# DEBUG!! - not working
# Find number of people in each age bin
# average_purch_total_per_person_df
age_counts_df = average_purch_total_df.groupby("Age Bins").count()
age_counts_df
#print(len(age_counts_df))
#print(len(average_purch_total_df["Price"]))
#avg_purch_df["Average Total Purchase Per Person"] = average_purch_total_df.groupby["Price"] / age_counts_df["Age"]
#avg_purch_df

# DEBUG - Not Working!! 1) Confirm calc method, 2) address columns and NaN
avg_purch_df["Average Total Purchase Per Person"] = total_purchase_df["Price"] / age_counts_df["Age"]
avg_purch_df

# Combine stats into 1 dataframe
purchase_stats_df["Average Purchase Price"] = average_purchase_df["Price"]
purchase_stats_df["Total_Purchase_value"] = total_purchase_df["Price"]
purchase_stats_df

avg_purch_df

       Purchase Counts
<10                 23
10-14               28
15-19              136
20-24              365
25-29              101
30-34               73
35-39               41
40+                 13


,Adairialis76,Adastirin33,Aeda94,Aela59,Aelaria33,Aelastirin39,Aelidru27,Aelin32,Aelly27,Aellynun67,...,Yastyriaphos75,Yasur35,Yasur85,Yathecal72,Yathecal82,Yathedeu43,Yoishirrala98,Zhisrisu83,Zontibe81,Average Total Purchase Per Person
Age Bins,,,,,,,,,,,,,,,,,,,,,
<10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.427500
10-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.869412
15-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.587667
20-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.258764
25-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.966102
30-34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.095238
35-39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.273929
40+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254933


In [72]:
#EIGHTH CELL
# Top Spenders
# Create a summary data frame with SN and Count of purchases
top_df = raw_data_df.groupby("SN").count()
top_df

#total_purchase_df = purchase_demo_df.groupby("Purchase Counts").sum()
# Add to summary data frame mean purchase value
top_mean_df = raw_data_df.groupby("SN").mean()
top_mean_df

# Add to summary data frame total purchase value
top_total_df = raw_data_df.groupby("SN").sum()
top_total_df

# create new df = just columns of interest
top_spenders_df["Purchase Count"] = top_df[["Age"]]
top_spenders_df

top_spenders_df["Average Purchase Price"] = top_mean_df["Price"]
top_spenders_df

top_spenders_df["Total Purchase Value"] = top_mean_df["Price"]
top_spenders_df

# Sort dataframe decending order
# DEBUG!! - sorting not working - error code
#top_spenders_df = top_spenders_df.sort_values("Total Purchase Value", axis=1, ascending=False)
#top_spenders_df

# Remove extra column

# Truncate summary data frame to 1ist first 5 records
# top_spenders_head_df = top_spenders_df.head()

,Age,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,,
Adairialis76,1,1,2.280000,2.280000
Adastirin33,1,1,4.480000,4.480000
Aeda94,1,1,4.910000,4.910000
Aela59,1,1,4.320000,4.320000
Aelaria33,1,1,1.790000,1.790000
...,...,...,...,...
Yathecal82,3,3,2.073333,2.073333
Yathedeu43,2,2,3.010000,3.010000
Yoishirrala98,1,1,4.580000,4.580000


In [70]:
#NINTH CELL
# Most Popular Items by Purchase Count
#Item ID
#Item Name
#Purchase Count
#Item Price
#Total Purchase Value

# Find count of items purchased
popular_items_df = raw_data_df.groupby("Item ID").count()
popular_items_df

# Find total purchase value
#pop_items_TPV_df = raw_data_df.groupby("Item ID").sum()
pop_items_TPV_df = raw_data_df[["Item ID", "Price"]].groupby("Item ID").sum()
pop_items_TPV_df

# Create datafraem with just item id, item name, and price
item_name_df = raw_data_df[["Item ID", "Item Name", "Price"]]
item_name_df = item_name_df.drop_duplicates()
item_name_df


# Combine data frome popular_items_df, pop_items_TPV_df, item_name_df into 1 dataframe
most_popular_items_df = item_name_df

most_popular_items_df["Purchase Count"] = pd.DataFrame(popular_items_df["Age"])
#most_popular_items_df = most_popular_items_df.rename(columns={"Age": "Purchase Count"})

#most_popular_items_df["Item Name"] = item_name_df["Item Name"]
most_popular_items_df["Total Purchase Value"] = pop_items_TPV_df["Price"]

# Reorder columns
most_popular_items_df = most_popular_items_df[["Item ID", "Item Name", "Purchase Count", "Price", "Total Purchase Value"]]
most_popular_items_df

# Resort Descending by Purchase Count
most_popular_items_df = most_popular_items_df.sort_values(["Purchase Count"], ascending=False)
most_popular_items_df


#df.sort_values(['a', 'b'], ascending=[True, False])
# DEBUG - Item name for Item ID 178 is NaN.  Why??? How to fix???
# DEBUG - Purchase Count = NaN for many
# DEBUG - Results do not match starter code



#DEBUG!! group by not working
#most_popular_items_df = most_popular_items_df.groupby("Item ID")

,Item ID,Item Name,Purchase Count,Price,Total Purchase Value
92,39,"Betrayal, Whisper of Grieving Widows",13.0,3.94,59.99
145,46,Hopeless Ebon Dualblade,9.0,1.33,41.22
132,145,Fiery Glass Crusader,9.0,4.58,28.99
108,85,Malificent Bag,9.0,1.75,31.77
34,160,Azurewrath,8.0,4.40,17.76
...,...,...,...,...,...
673,130,Alpha,NaN,2.07,NaN
700,90,Betrayer,NaN,2.94,NaN
717,177,"Winterthorn, Defender of Shifting Worlds",NaN,2.08,NaN
727,104,Gladiator's Glaive,NaN,1.93,NaN


In [71]:
#TENTH CELL
# Most Profitable Items

# Resort Descending by Total Purchase Value
most_popular_items_df = most_popular_items_df.sort_values(["Total Purchase Value"], ascending=False)
most_popular_items_df

#DEBUG - NaN values in Total Purchase Value, Purchase Count
#DEBUG - Results do not match starter code

,Item ID,Item Name,Purchase Count,Price,Total Purchase Value
92,39,"Betrayal, Whisper of Grieving Widows",13.0,3.94,59.99
145,46,Hopeless Ebon Dualblade,9.0,1.33,41.22
59,105,Hailstorm Shadowsteel Scythe,8.0,3.03,33.84
108,85,Malificent Bag,9.0,1.75,31.77
78,50,Dawn,7.0,4.60,30.80
...,...,...,...,...,...
673,130,Alpha,NaN,2.07,NaN
700,90,Betrayer,NaN,2.94,NaN
717,177,"Winterthorn, Defender of Shifting Worlds",NaN,2.08,NaN
727,104,Gladiator's Glaive,NaN,1.93,NaN
